In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/Advertising.csv", index_col=0)

X, y = df.drop("Sales", axis="columns"), df["Sales"]

model_polynomial = PolynomialFeatures(3, include_bias=False)
poly_feature = model_polynomial.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(poly_feature, y, test_size=0.33, random_state=1337)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((134, 19), (66, 19), (134,), (66,))

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)



print(f"{scaled_X_train.mean():.2f}, {scaled_X_test.mean():.2f}")     # Så nära 0 som möjligt är bra

# Datan är nu skalad för att få std (standard deviation) = 1

-0.00, 0.03


In [40]:
# Ridge regression för att minska variansen

from sklearn.linear_model import Ridge 
from sklearn.metrics import mean_absolute_error, mean_squared_error

# L2 regularization

# alpha = lambda 

model_ridge = Ridge(alpha=0.1)
model_ridge.fit(scaled_X_train, y_train) # .fit innebär att man tränar algoritmen på datan

# Y_hat = Y_pred
y_hat = model_ridge.predict(scaled_X_test) # Y_hat = en vektor på förutsägelsen

MSE = mean_squared_error(y_test, y_hat) # Avståndet mellan  y_test och y_hat i kvadrat, (felet)
RMSE = np.sqrt(MSE) # Units, i detta exempel en halv unit i fel


MSE, RMSE





(0.24959538036150153, 0.49959521651182925)

In [41]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha=0.1)
model_lasso.fit(scaled_X_train, y_train)
Y_hat = model_lasso.predict(scaled_X_test)

print(model_lasso.coef_)

np.sqrt(mean_squared_error(y_test, Y_hat))

[ 1.77053674  0.16177077  0.         -0.          3.77413423  0.
  0.          0.04720898  0.         -0.37585383 -0.         -0.
  0.          0.          0.          0.          0.          0.
  0.        ]


0.7738198161795438

In [42]:
from sklearn.linear_model import RidgeCV

model_ridgeCV = RidgeCV(alphas=[.0001, .001, .01, .1, .5, 1, 5, 10], scoring="neg_mean_squared_error")

model_ridgeCV.fit(scaled_X_train, y_train)

# Score ska vara högt, Hur mycket av variansen som förklaras av modellen
# 99% av våra värden ligger innanför
print(model_ridgeCV.coef_, "\n \n", model_ridgeCV.score(scaled_X_train, y_train)) 
model_ridgeCV.alpha_

# Våran träningsdata är väldigt varierande eftersom om vi ändrar randomstate i början så blir resultatet här 
# på vilken alpha  som är bäst väldigt annorlunda 

# Denna polynomiella funktion 

[  7.34985134   0.6331339    0.4575767  -10.6664558    4.69979729
  -1.21795087  -0.33002651  -0.2049405   -0.18235904   5.19374784
  -1.37857412   1.00487749   0.51536908  -0.39391912   0.23265959
  -0.28009281   0.38741237   0.14013473  -0.09899025] 
 
 0.9912715899638569


0.01

In [43]:
from sklearn.linear_model import LassoCV

model_lassoCV = LassoCV(eps=0.001, n_alphas= 100, max_iter=10000, cv=5)
model_lassoCV.fit(scaled_X_train, y_train)

print(f"alpha (lambda) = {model_lassoCV.alpha_}")

y_hat = model_lassoCV.predict(scaled_X_test)

np.sqrt(mean_squared_error(y_test, y_hat)) # RMSE

alpha (lambda) = 0.004956246150210799


0.4529065286091838

In [52]:
from sklearn.linear_model import ElasticNetCV

# eps = "steglängden", epsilon. 

model_elasticCV = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1, 1], eps=0.001, n_alphas=100, max_iter=10000, cv=5)
model_elasticCV.fit(scaled_X_train, y_train)

print(f"l1 ratio = {model_elasticCV.l1_ratio_} : alpha (lambda) = {model_elasticCV.alpha_}")

y_hat = model_elasticCV.predict(scaled_X_test) # prediktering med våran tränade modell "model_elasticCV"

np.sqrt(mean_squared_error(y_test, y_hat))

# Testar alla l1_ratio och vi kom fram till med kors-valideringen i elasticnetCV att 1.0 varden bästa ration för modellen

# Vi tar allt från lasso i detta fall, elastic net testar båda. Hade vi fått l1 = 0.5 hade vi tagit hälften av varje modell (ridge, lasso)

l1 ratio = 1.0 : alpha (lambda) = 0.004956246150210799


0.4529065286091838